In [56]:
# import necessary packages
import sys
import torch 
import evaluate
from transformers import (AutoTokenizer,
                          AutoModelForSequenceClassification,
                          DataCollatorWithPadding,
                          get_scheduler)
from datasets import load_dataset
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm.auto import tqdm

sys.path.append('../')

# custom imports
from utils.GetLowestGPU import GetLowestGPU

device = GetLowestGPU()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Device set to cuda:2


# Data Loading / Tokenization

In [57]:
# load dataset
raw_datasets = load_dataset("glue", "mrpc")

# preprocess the data
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# define tokenize function to use with mapping
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

# tokenize the datasets
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# instantiate collator
data_collator = DataCollatorWithPadding(tokenizer)

/mnt/DGX01/Personal/krusepi/.venv/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# Other Preprocessing

In [58]:
# remove columns corresponding to values that the model does not expect
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])

# rename label column to labels
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

# set dataset format to pytorch
tokenized_datasets.set_format("torch")

# print col names to check
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

# Fine-Tuning

In [59]:
# define dataloaders
train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, collate_fn=data_collator, batch_size=8
)

eval_dataloader = DataLoader(
    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=8
)

In [60]:
# inspect example batch
for batch in train_dataloader:
    break
{k: v.shape for k, v  in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 81]),
 'token_type_ids': torch.Size([8, 81]),
 'attention_mask': torch.Size([8, 81])}

In [61]:
# instantiate model
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# sample forward pass
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor(0.6780, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


In [62]:
# initialize optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# and scheduler
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

print(num_training_steps)

1377


In [63]:
# map model to device
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [69]:
# set progress bar
progress_bar = tqdm(range(num_training_steps))

# eval loop
metric = evaluate.load("glue", "mrpc")

# loop through epochs
for epoch in range(num_epochs):

    # set model to train mode
    model.train()

    for batch in train_dataloader:

        # grab batch and map to device
        batch = {k: v.to(device) for k, v in batch.items()}

        # forward pass
        outputs = model(**batch)
        loss = outputs.loss

        # backward pass
        loss.backward()

        # update optimizer
        optimizer.step()

        # update scheduler
        lr_scheduler.step()

        # zero gradients
        optimizer.zero_grad()

        # update progress bar
        progress_bar.update(1)

    # set to eval mode
    model.eval()

    for batch in eval_dataloader:

        # get batch
        batch = {k: v.to(device) for k, v in batch.items()}

        # forward pass
        with torch.no_grad():
            outputs = model(**batch)

        # get logits, predictions
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        metric.add_batch(predictions=predictions, references=batch["labels"])

    print(metric.compute())

  0%|          | 0/1377 [00:00<?, ?it/s]

{'accuracy': 0.8553921568627451, 'f1': 0.8991452991452993}
{'accuracy': 0.8553921568627451, 'f1': 0.8991452991452993}
{'accuracy': 0.8553921568627451, 'f1': 0.8991452991452993}


{'accuracy': 0.8553921568627451, 'f1': 0.8991452991452993}